 # Optimization Methods

 Rock Boynton | CS 4850

 ## Introduction

 In this notebook, we will practice writing cost functions for optimization problems using an iterative optimization algorithm we developed.
 We will also find and distinguish between global and local optima, and then play with the initial conditions to see how they affect the result.

 We will then run experiments on the following models:

 1. Cubic Model

 2. Quartic Model

 3. Gaussian Model

 ## Summary of Results

Experiment 1 showed us how our algorithm can work as expected to find the minimum of a function, but also showed up how much the starting parameters matter. If you choose poorly you could end up chasing infinity along a gradient, never finding an optima. Experiment 3 again showed us how the starting point matters, but this time in finding *global* vs *local* optima. Experiment 3 showed us how, more often than not, we are trying to optimize hyper-parameters of a model, and using something like mean-squared-error instead of a function to model data itself. That was tricky to determine, though.

 ---

In [ ]:
import optim

import numpy as np 
import matplotlib.pyplot as plt

 ## Experiment 1: Cubic Model

In [ ]:
class CubicCostFunction:
    def cost(self, params):
        """
        Implements the curve:
        
        y = x**3 - 3*x**2 - 144*x + 432
        """
        return params**3 - 3*params**2 - 144*params + 432

NUMERIC_DIFF_DELTA = 1e-5
GRADIENT_TOL = 1e-2
STEP_SIZE = 0.01
MAX_ITER = 100
TOL = 1e-5

optimizer = optim.Optimizer(STEP_SIZE, MAX_ITER, GRADIENT_TOL, NUMERIC_DIFF_DELTA)
cost = CubicCostFunction()

x_wide = np.arange(-30, 30, 0.01)
x_narrow = np.arange(-8, 15, 0.01)

for x_range in (x_wide, x_narrow):
    gradient = np.array([optimizer._gradient(cost, np.array([x])) for x in x_range])
    plt.plot(x_range, cost.cost(x_range), x_range, gradient)
    for starting_params in (25, 0, -25):
        optimized_params, iters = optimizer.optimize(cost, np.array([starting_params], dtype=np.float64))
        print(f'Found min at {optimized_params} starting at {starting_params} in {iters} iterations of optimization algorithm.')
        params = np.array([starting_params, optimized_params])
        plt.plot(params, cost.cost(params), 'ro-', scalex=False, scaley=False)
    plt.legend(['cost function', 'gradient'])
    plt.show()
 
# Notice that x is at a local optima where the derivitive == 0

 ## Experiment 2: Quartic Model

In [ ]:
class QuarticCostFunction:
    def cost(self, params):
        """
        Implements the curve:
        
        y = 3*x**4 - 16*x**3 - 18*x**2
        """
        return 3*params**4 - 16*params**3 - 18*params**2

MAX_ITER = 1000
STEP_SIZE = 0.001
GRADIENT_TOL = 1e-5
NUMERIC_DIFF_DELTA = 1e-5

optimizer = optim.Optimizer(STEP_SIZE, MAX_ITER, GRADIENT_TOL, NUMERIC_DIFF_DELTA)
cost = QuarticCostFunction()

x_range = np.arange(-3, 6.75, 0.01)

# plt.plot(x_range, cost.cost(x_range), x_range, optimizer._gradient(cost, x_range)) # doesn't work
gradient = np.array([optimizer._gradient(cost, np.array([x])) for x in x_range])
plt.plot(x_range, cost.cost(x_range), x_range, gradient)
for starting_params in (6, 2, -2):
    optimized_params, iters = optimizer.optimize(cost, np.array([starting_params], dtype=np.float64))
    print(f'Found min at {optimized_params} starting at {starting_params} in {iters} iterations of optimization algorithm.')
    params = np.array([starting_params, optimized_params])
    plt.plot(params, cost.cost(params), 'ro-', scalex=False, scaley=False)
plt.legend(['cost function', 'gradient'])
plt.show()
 
# Notice that x is at a local optima where the derivitive == 0

 ## Experiment 3: Gaussian Model

In [ ]:
from math import pi, e, sqrt

class GaussianCostFunction:
    def __init__(self, data):
        self.data = data


    def _calculate_y(self, x, mu, sigma):
        """
        Implements the Gaussian function
        """
        return (1 / (sigma*sqrt(2*pi))) * e**(-0.5 * ((x - mu) / sigma)**2)


    def cost(self, params):
        """
        Implements the mean squared error of the gaussian function
        params[0] = mu
        params[1] = sigma
        """
        return np.mean((self.data[:, 1] - self._calculate_y(self.data[:, 0], params[0], params[1])) ** 2)

MAX_ITER = 1000
STEP_SIZE = 1
GRADIENT_TOL = 1e-5
NUMERIC_DIFF_DELTA = 1e-5

datapath = "gaussdist.csv"
data = np.loadtxt(datapath, delimiter=',')
data = data[:, [1, 0]] # flip columns 

cost = GaussianCostFunction(data)

x_range = np.arange(-10, 10, 0.01)

starting_params = [1, 0.75] # mu, sigma
plt.plot(x_range, cost._calculate_y(x_range, starting_params[0], starting_params[1]), color='red')
plt.scatter(data[:, 0], data[:, 1])
plt.legend(['unoptimized model', 'data'])
plt.show()

optimizer = optim.Optimizer(STEP_SIZE, MAX_ITER, GRADIENT_TOL, NUMERIC_DIFF_DELTA)
optimized_params, iters = optimizer.optimize(cost, np.array(starting_params, dtype=np.float64))

print(f'Found min at {optimized_params} starting at {starting_params} in {iters} iterations of optimization algorithm.')

plt.plot(x_range, cost._calculate_y(x_range, optimized_params[0], optimized_params[1]), color='red')
plt.scatter(data[:, 0], data[:, 1])
plt.legend(['optimized model', 'data'])
plt.show()

 ## Questions

 1. For experiment 1, what were the solutions for steps 5, 6, and 7? Describe any
 differences that you saw. How do you explain these differences?
    - Found min at [ 8.01324051] starting at 25 in 9 iterations of optimization algorithm.
    - Found min at [ 7.99087673] starting at 0 in 15 iterations of optimization algorithm.
    - Found min at [ -3.60287971e+19] starting at -25 in 8 iterations of optimization algorithm.
    - Starting at 25 and 0 it makes sense that the algorithm would "follow the gradient" and find the min at around 8, but when starting at -25, it follows the gradient down to infinity hence the huge negative value for the calculated min.

 2. For experiment 2, what were the solutions for steps 5, 6, and 7? Describe any
 differences that you saw. Relate your knowledge of global and local optima to this
 discussion. What is the correct solution and why?
   - Found min at [ 4.64576556] starting at 6 in 30 iterations of optimization algorithm.
   - Found min at [ 4.64572556] starting at 2 in 41 iterations of optimization algorithm.
   - Found min at [-0.64598601] starting at -2 in 176 iterations of optimization algorithm.
   - When starting at 6 and 2, the gradient points toward the global min of about 4.646, but when starting at -2, the gradient points toward the local min of about -0.646. The correct solution is the global min, because that is where the function is truly minimized

 2. For experiment 3, what do you observe when comparing your model with fit parameters
 to your data. If you were to get a new set of independent variables, how could you use
 this model and what information would it give you?
  - When comparing the model with with fit parameters (mu = 2.9997177, sigma = 0.99997786), I see that the model lines up nicely with the curve of the dataset. If you got a new set of independant data, you could create a new cost object with the data then use the optimized mu and sigma to plot it with a scatter of the data to see how well the mean and standard deviation fit the new data.
